In [269]:
import pickle
with open('../data/data.pickle', 'rb') as f:
  data = pickle.load(f)


In [270]:
texts = data['fixcapital']

In [271]:
def flat(l):
    for k in l:
        if not isinstance(k, (list, tuple)):
            yield k
        else:
            yield from flat(k)
texts = list(flat(texts))

In [272]:
print(str(len(texts)) + ' texts')

38650 texts


First step: Deal with password without capital words

In [273]:
from sklearn.feature_extraction.text import TfidfVectorizer

# change the max_df to get different results

tfidf_vectorizer = TfidfVectorizer(max_df=0.9999, max_features=200000,
                                   min_df=0.0001, stop_words=my_stop_words, 
                                   use_idf=True, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

print(tfidf_matrix.shape)


CPU times: user 361 ms, sys: 8.19 ms, total: 369 ms
Wall time: 368 ms
(38650, 3652)


In [200]:
terms = tfidf_vectorizer.get_feature_names()

In [253]:
from sklearn.cluster import KMeans
# num of clusters depends on dataset 
num_clusters = 5
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

CPU times: user 26.2 s, sys: 18 ms, total: 26.2 s
Wall time: 4.37 s


Try to get a matrix

In [215]:
import pandas as pd
df_tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray())


In [236]:
import pandas as pd
import numpy as np

data = pd.read_csv('../data/38650-username-sktorrent.txt', encoding='ISO-8859-1', header=None)
import re

data.columns = ['original']
    
data['original'] = data['original'].astype(str)




In [246]:
data = data['original'][0:38650]


In [254]:
news = {'username': data,'text': texts,'cluster': clusters}

frame = pd.DataFrame(news, index = [clusters], columns = ['date','text','cluster'])

In [262]:
frame['cluster'].value_counts() #number of films per cluster (clusters from 0 to 4)
order_centroids[i, :6]

array([1786, 2570,  470,  545, 1404, 2663])

In [264]:
print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :6]:
        print(' %s' % terms[ind].split(' '), end=',')
    print()
    print()

Top terms per cluster:

Cluster 0 words: ['123'], ['ka'], ['to'], ['11'], ['12'], ['in'],

Cluster 1 words: ['tek'], ['kr', 'tek'], ['koko', 'tek'], ['kr'], ['koko'], ['023'],

Cluster 2 words: ['ko'], ['ko', '123'], ['in', 'ko'], ['martin', 'ko'], ['sam'], ['sam', 'ko'],

Cluster 3 words: ['ik'], ['18'], ['ik', '123'], ['as'], ['rom'], ['last'],

Cluster 4 words: ['kr'], ['pec'], ['alice'], ['at'], ['in'], ['pole'],



In [180]:
coun = 0
for i in clusters:
    if(i == 0):
        coun = coun+1
coun

974

In [268]:
for i in df_tfidf_matrix[0][1:10]:
    if(i!=0):
        print(i)

In [19]:
news = {'date': dates,'articlecode': articlecodes, 'title': titles,'text': texts,'cluster': clusters}
frame = pd.DataFrame(news, index = [clusters], columns = ['date','articlecode','title','text','cluster'])

Top terms per cluster:

Cluster 0 words:

NameError: name 'vocab_frame' is not defined

https://github.com/llSourcell/Latent_Dirichlet_Allocation/blob/master/NewAnalysis-Normal.ipynb